In [1]:
import os
import sys
import pathlib

import yaml
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

    
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
import matplotlib.cm

cm = plt.get_cmap('bwr').copy()
cm.set_bad('k', alpha=0.1)

from bci_plot.metadata import sessions_info_w_day
from bci_plot.metadata import layouts
layout = layouts.layouts['wet64']
from bci_plot.gen_data.grad import instantiate
from bci_plot.gen_data.grad.EEGNet_grad import EEGNet

from bci_plot.gen_fig.grad import gen_grad_fig

/home2/johannes/bci_plot/bci_plot/gen_fig/grad/gen_grad_fig.py:27: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  _ = info.set_montage(montage)


In [2]:
sessions, decoders, folds, subjects, days, *_ = list(zip(*sessions_info_w_day.sessions_info))
doubles = sorted(list(set(list(zip(subjects, decoders)))))
triplets = sorted(list(set(list(zip(subjects, decoders, days)))))

In [4]:
# split by user
# 4 plots only.

for subject_u in sorted(set(subjects)):
    sessions_compile = [session for (session, subject, decoder, day) in zip(sessions, subjects, decoders, days) if (subject,) == (subject_u,)]
    
    grad_data_compile = [np.load(f'../../data/grad_vals/{session}.npz') for session in sessions_compile]
    F1 = grad_data_compile[0]['conv_weights'].shape[0]
    D = grad_data_compile[0]['depthwise_weights'].shape[0] / F1
    D = int(D)
    Ls = [grad_data['L'] for grad_data in grad_data_compile]
    LL = sum(Ls)

    channel_scale_grad = np.zeros_like(grad_data_compile[0]['channel_scale_grad'])

    for session, grad_data in zip(sessions_compile, grad_data_compile):
        channel_scale_grad += (grad_data['L']/LL)*grad_data['channel_scale_grad']

    # plot stuff
    path = f'../../figs/grad_figs/by_user/{subject_u}_grad.pdf'
    
    gen_grad_fig.gen_topo_partial(channel_scale_grad, path)
    plt.close()